In [1]:
import pandas as pd
import numpy as np
import fastparquet as fp
import matplotlib.pyplot as plt
import sklearn.model_selection as skms
import sklearn.metrics as skme
import random
import re
import src.helper.helper as hlp
import src.helper.const as const

In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ths/miniconda3/envs

In [2]:
# Config
FILE_DIR = "./"
DATA_DIR = FILE_DIR + "../data/interim/"
POSTER_DIR = DATA_DIR + "posters/"

In [3]:
# Init
pd.set_option('display.max_colwidth', None)
GENRE_COLS = const.GENRE_OHE_COLS

In [73]:
# Load train-eval-test splits
df_ori = pd.read_parquet(DATA_DIR + 'df_cleaned.gzip')
df = pd.read_parquet(DATA_DIR + 'df_train_v1.gzip')
df_train = pd.read_parquet(DATA_DIR + 'df_train-split_v1.gzip')
df_eval = pd.read_parquet(DATA_DIR + 'df_eval-split_v1.gzip')
df_test = pd.read_parquet(DATA_DIR + 'df_test-split_v1.gzip')

# Set true labels
y_true_test = df_test[GENRE_COLS].to_numpy()
y_true_train = df_train[GENRE_COLS].to_numpy()

In [74]:
def accuracy_score(y_test: np.array, y_pred: np.array, verbose: bool = False):
    test = np.array(y_test, dtype=int)
    pred = np.array(y_pred, dtype=int)
    value = 0
    pcount = 0

    # Fetch data
    for i in range(0, len(y_test)):
        bws = np.bitwise_and(test[i], pred[i]).sum()
        mas = max(test[i].sum(), pred[i].sum())
        res = bws // mas
        if verbose:
            if res > 0:
                pcount += 1
                print(pcount, ". ", test[i], "&", pred[i],
                      "=", bws, "/", mas, "->", res)
        value += res

    # Calc accuracy
    acc = value / len(test)
    if verbose:
        print(f"{value} / {len(test)} = {acc}")
    else:
        print(acc)

# Random Guess using one label per movie

## Test datasets

In [71]:
# Random guess using multi-classes (one label per movie)
l_y_pred = []
l_gen_art = []
for i in range(0, len(df_test)):
    gen = np.zeros(len(GENRE_COLS))
    pos = np.random.randint(0, len(GENRE_COLS))
    gen[pos] = 1
    l_gen_art.append(gen)
y_pred_test = np.array(l_gen_art)
accuracy_score(y_true_test, y_pred_test, False)

0.05357628501495785


## Training datasets

In [75]:
# Random guess using multi-classes (one label per movie)
l_y_pred = []
l_gen_art = []
for i in range(0, len(df_train)):
    gen = np.zeros(len(GENRE_COLS))
    pos = np.random.randint(0, len(GENRE_COLS))
    gen[pos] = 1
    l_gen_art.append(gen)
y_pred_train = np.array(l_gen_art)
accuracy_score(y_true_train, y_pred_train, False)

0.05210812648758926


# Random Guess using up to 3 labels per movie

## Test datasets

In [77]:
# Random guess using multi-labels following the distribution (up to 3 labels per movie)
l_count_gen = hlp.get_dist_of_simple_genre_combis(df_test, const.GENRE_OHE_COLS, True)
l_gen_art = []
np.random.seed(const.SEED)
for num_gen, count_gen in enumerate(l_count_gen):
    for count in range(0, count_gen):
        gen = np.zeros(len(GENRE_COLS))
        for i in range(0, num_gen):
            pos = np.random.randint(0, len(GENRE_COLS))
            gen[pos] = 1
        l_gen_art.append(gen)
df_tmp = pd.DataFrame(l_gen_art)
y_pred_test = df_tmp.to_numpy()
accuracy_score(y_true_test, y_pred_test, False)

Number of movies holding 0 genres: 0
Number of movies holding 1 genres: 3553
Number of movies holding 2 genres: 124
Number of movies holding 3 genres: 0
Number of movies holding 4 genres: 0
Number of movies holding 5 genres: 0
Number of movies holding 6 genres: 0
Number of movies holding 7 genres: 0
Number of movies holding 8 genres: 0
Number of movies holding 9 genres: 0
Number of movies holding 10 genres: 0
Number of movies holding 11 genres: 0
Number of movies holding 12 genres: 0
Number of movies holding 13 genres: 0
Number of movies holding 14 genres: 0
Number of movies holding 15 genres: 0
Number of movies holding 16 genres: 0
Number of movies holding 17 genres: 0
Number of movies holding 18 genres: 0
0.04813706826217025


## Training datasets

In [76]:
# Random guess using multi-labels following the distribution (up to 3 labels per movie)
l_count_gen = hlp.get_dist_of_simple_genre_combis(df_train, const.GENRE_OHE_COLS, True)
l_gen_art = []
np.random.seed(const.SEED)
for num_gen, count_gen in enumerate(l_count_gen):
    for count in range(0, count_gen):
        gen = np.zeros(len(GENRE_COLS))
        for i in range(0, num_gen):
            pos = np.random.randint(0, len(GENRE_COLS))
            gen[pos] = 1
        l_gen_art.append(gen)
df_tmp = pd.DataFrame(l_gen_art)
y_pred_train = df_tmp.to_numpy()
accuracy_score(y_true_train, y_pred_train, False)

Number of movies holding 0 genres: 0
Number of movies holding 1 genres: 11382
Number of movies holding 2 genres: 375
Number of movies holding 3 genres: 7
Number of movies holding 4 genres: 0
Number of movies holding 5 genres: 0
Number of movies holding 6 genres: 0
Number of movies holding 7 genres: 0
Number of movies holding 8 genres: 0
Number of movies holding 9 genres: 0
Number of movies holding 10 genres: 0
Number of movies holding 11 genres: 0
Number of movies holding 12 genres: 0
Number of movies holding 13 genres: 0
Number of movies holding 14 genres: 0
Number of movies holding 15 genres: 0
Number of movies holding 16 genres: 0
Number of movies holding 17 genres: 0
Number of movies holding 18 genres: 0
0.04981298877932676
